# License

    Jupyter notebook for accessing CUDA
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
from matplotlib import pyplot as plt

import IPythonMagic
from Timer import Timer

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

In [2]:
%setup_logging
%cuda_context_handler context

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10328 / 11441 MB available
Created context handle <34121840>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Simone_Carriero/MilanoGPU2018/notebooks/cuda_cache


In [12]:
kernel_src = """

__global__ void shmemReduction(float *output, float *input, int size)
{
    float max_value = - 99999999;
    for (int i = threadIdx.x; i < size; i += blockDim.x)
    {
        max_value = fmaxf(max_value, input[i]);
    }
    __shared__ float max_shared[64];
    max_shared[threadIdx.x] = max_value;
    
    __syncthreads();
    
    if(threadIdx.x < 32)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 32]);
    __syncthreads();
    if(threadIdx.x < 16)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    if(threadIdx.x < 8)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]);
    if(threadIdx.x < 4)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]);
    if(threadIdx.x < 2)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]);
    if(threadIdx.x < 1)
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]);
    
    if(threadIdx.x == 0)
        output[0] = max_shared[0];
}
"""
kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

In [13]:
n = 64;
#a = np.random.random((1,n)).astype(np.float32)
a = np.arange(n).astype(np.float32)
print(a)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 32
b = np.empty((1, num_threads)).astype(np.float32)

b_g = GPUArray(b.shape, b.dtype)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63.]


In [14]:
block_size = (num_threads, 1, 1)
grid_size = (block_size[0] // num_threads, 1, 1)

kernel_function(b_g, a_g, np.int32(n), grid=grid_size, block=block_size)

b_g.get(b)

print(a)
print(b)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63.]
[[53.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
  18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]]
